In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf


def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def getCIFAR10(direc,test):
    # Converts the data in batches to a full training set
    if test:
        dic = unpickle(direc)
        x_test = dic[b'data']
        y_test = dic[b'labels']
        return np.asarray(x_test), np.asarray(y_test)
    else:
        for j in range(1, 6):
            file = direc + str(j)
            dic = unpickle(file)
            if j == 1:
                x_train = dic[b'data']
                y_train = dic[b'labels']
            else:
                temp_x = dic[b'data']
                temp_y = dic[b'labels']
                x_train = np.concatenate((x_train, temp_x))
                y_train = np.concatenate((y_train, temp_y))
        return x_train, y_train

In [2]:
direc = 'cifar-10-batches-py/data_batch_'
x_train, y_train = getCIFAR10(direc, False)
x_train= x_train.astype(np.float64)
y_train = y_train.astype(np.int64)
print(x_train.shape)
print(y_train.shape)

direc = 'cifar-10-batches-py/test_batch'
x_test, y_test = getCIFAR10(direc, True)
x_test= x_test.astype(np.float64)
y_test = y_test.astype(np.int64)
print(x_test.shape)
print(y_test)

FileNotFoundError: ignored

In [0]:
names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
for label in range (0, 10):
    idx = []
    for i in range (y_train.shape[0]):
        if y_train[i] == label:
            idx.append(i)
            break
for i, ax in enumerate(axes.flat):
  if i < 10:
    # Get the weights for the i'th digit and reshape it.
    # Note that w.shape == (img_size_flat, 10)
    img = x_train[idx[i-1]].reshape(3,32,32).transpose([1, 2, 0])
    ax.set_xlabel(names[y_train[idx[i-1]]])
    # Plot the image.
    ax.imshow(img)
    # Remove ticks from each sub-plot.
    ax.set_xticks([])
    ax.set_yticks([])

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 32, 32, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  accuracy = tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  eval_metric_ops = {'accuracy': accuracy}
  tf.summary.scalar('accuracy', accuracy[1])

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)

  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [0]:
def main(unused_argv):
    # Load training and eval data
    #mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    #train_data = mnist.train.images  # Returns np.array
    #print(train_data.shape)
    #train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    #eval_data = mnist.test.images  # Returns np.array
    #eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    #train_data = x_train
    train_labels = y_train
    #eval_data = x_test
    eval_labels = y_test
    train_data = None
    train_data = (x_train[:,:32*32]+x_train[:,32*32:32*32*2]+x_train[:,32*32*2:])/3
    eval_data = (x_test[:,:32*32]+x_test[:,32*32:32*32*2]+x_test[:,32*32*2:])/3
    
    #print(train_labels)
    # Create the Estimator
    cifar10_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/cfar10_test3")

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
    cifar10_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = cifar10_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)


if __name__ == "__main__":
    tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_is_chief': True, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_train_distribute': None, '_task_id': 0, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8ce1e20d68>, '_master': '', '_session_config': None, '_model_dir': '/tmp/cfar10_test3', '_save_summary_steps': 100, '_service': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_task_type': 'worker'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/cfar10_test3/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkp

INFO:tensorflow:step = 2, loss = 14.21654987335205
INFO:tensorflow:probabilities = [[0.09425268 0.18361029 0.03286688 0.00528017 0.00212938 0.20870912
  0.0102107  0.03160191 0.08708984 0.34424902]
 [0.00728219 0.00171193 0.09353164 0.11049824 0.05612991 0.12677025
  0.58336211 0.00671501 0.00643504 0.00756369]
 [0.00042741 0.00698317 0.01648822 0.00040758 0.01386749 0.89827952
  0.02427454 0.00949028 0.00081381 0.028968  ]
 [0.06958875 0.55066649 0.01585947 0.02761608 0.00238775 0.0550464
  0.05448916 0.03206643 0.18332624 0.00895323]
 [0.13343323 0.00814291 0.22818808 0.06614576 0.1253351  0.1206136
  0.06760554 0.0261893  0.03359639 0.19075009]
 [0.01817224 0.00216739 0.0022918  0.00400034 0.00962972 0.00459368
  0.00011102 0.95343314 0.0009156  0.00468506]
 [0.0039028  0.28878443 0.06030846 0.28757655 0.01278141 0.00406358
  0.0470672  0.01154256 0.2232382  0.06073482]
 [0.02200987 0.13284117 0.02485931 0.1632896  0.00160818 0.01343156
  0.03816058 0.56119282 0.03194911 0.01065779]

INFO:tensorflow:global_step/sec: 1.55183
INFO:tensorflow:probabilities = [[0.00430457 0.00295496 0.02667101 0.02455915 0.05964371 0.00081075
  0.06263193 0.00948293 0.05241301 0.75652798]
 [0.19834236 0.07378931 0.05573817 0.00420887 0.04130907 0.0826989
  0.11058862 0.33265686 0.06314075 0.03752711]
 [0.07908894 0.07953723 0.01890977 0.16063981 0.00786662 0.0159182
  0.05360074 0.26873894 0.05994087 0.25575888]
 [0.01912011 0.11012532 0.01843502 0.05480018 0.02303405 0.55019789
  0.08532503 0.10217174 0.02235038 0.01444028]
 [0.13623546 0.00027075 0.10405131 0.0034381  0.57768572 0.03261786
  0.01200387 0.00044523 0.02610008 0.10715163]
 [0.04946135 0.03452962 0.01973164 0.06468793 0.35565227 0.01856097
  0.01433539 0.01814208 0.11146818 0.31343058]
 [0.0639784  0.01941295 0.20481539 0.06153864 0.06781163 0.39494333
  0.00476222 0.0462461  0.12711029 0.00938105]
 [0.08611647 0.02988423 0.14630846 0.03743561 0.1069766  0.19862074
  0.10646906 0.20074393 0.04488424 0.04256067]
 [0.11185

INFO:tensorflow:step = 102, loss = 2.596475124359131 (64.441 sec)
INFO:tensorflow:probabilities = [[0.0175409  0.23049543 0.2851175  0.01977417 0.07151366 0.20354389
  0.02019101 0.00502639 0.05672933 0.09006772]
 [0.02267283 0.05724289 0.02172095 0.05984091 0.03266185 0.04063712
  0.03700528 0.19636375 0.44549954 0.08635487]
 [0.18619048 0.05548036 0.06684502 0.03631551 0.0716208  0.14710189
  0.03949786 0.22898344 0.06841301 0.09955162]
 [0.1761741  0.28591854 0.02366128 0.01348309 0.04021283 0.09056223
  0.01284245 0.03244895 0.12469441 0.20000211]
 [0.03320013 0.08718192 0.03163004 0.03277464 0.03052431 0.48203643
  0.02561776 0.18985639 0.05562147 0.03155691]
 [0.15639198 0.04547479 0.12968137 0.14853682 0.07259967 0.02753616
  0.12436327 0.07948709 0.14723119 0.06869766]
 [0.05974771 0.03351047 0.04286061 0.05267597 0.21850482 0.02668046
  0.4387254  0.03901262 0.074242   0.01403994]
 [0.07063447 0.00922881 0.35318281 0.03089068 0.0508758  0.13841513
  0.18668917 0.05735876 0.079

INFO:tensorflow:global_step/sec: 1.56233
INFO:tensorflow:probabilities = [[0.3946523  0.03040347 0.01201106 0.04854185 0.04545908 0.05867947
  0.03316201 0.08711019 0.05828317 0.23169739]
 [0.053605   0.14785566 0.05246386 0.06684668 0.02976508 0.22857488
  0.12419615 0.10834459 0.08241751 0.1059306 ]
 [0.27015723 0.05149244 0.06490409 0.06596797 0.05964646 0.04319072
  0.03991892 0.121911   0.11084938 0.17196178]
 [0.09727882 0.04623613 0.09131905 0.11572206 0.08795176 0.03440453
  0.17430341 0.05605441 0.23627156 0.06045827]
 [0.04723924 0.21524661 0.01493304 0.07974233 0.07259917 0.0089942
  0.00839592 0.02436744 0.21996157 0.30852046]
 [0.17642613 0.08968586 0.05974352 0.07730288 0.02879829 0.21879541
  0.02130083 0.14136722 0.10049701 0.08608284]
 [0.23120059 0.0147147  0.12783591 0.01991584 0.09832546 0.05391277
  0.23429488 0.06928061 0.10267917 0.04784007]
 [0.13091363 0.06313583 0.14329475 0.09240931 0.13705993 0.11215991
  0.1731977  0.02574206 0.05515702 0.06692985]
 [0.0811

...

INFO:tensorflow:step = 7402, loss = 1.8525832891464233 (63.701 sec)
INFO:tensorflow:probabilities = [[0.01936799 0.28999394 0.03756731 0.14407652 0.01012129 0.03430817
  0.10710527 0.09136666 0.05647465 0.2096182 ]
 [0.09139818 0.04338688 0.21070818 0.15524689 0.07549859 0.06984138
  0.07958397 0.11914125 0.10759243 0.04760225]
 [0.08749682 0.16614321 0.03081518 0.15086066 0.02509666 0.01750833
  0.02631262 0.10364042 0.16101638 0.23110971]
 [0.09522334 0.06416852 0.1077459  0.06844237 0.15337326 0.12710044
  0.12860183 0.09580786 0.1164582  0.04307828]
 [0.12819486 0.07479943 0.05562358 0.06890102 0.0178001  0.01677093
  0.05432619 0.02833719 0.40237629 0.1528704 ]
 [0.0647587  0.02462305 0.07174115 0.06144158 0.06445231 0.07102222
  0.08680432 0.05449188 0.37302523 0.12763956]
 [0.06055843 0.22841514 0.00165188 0.02025964 0.01105114 0.00228362
  0.09563785 0.02550237 0.01357946 0.54106045]
 [0.0576651  0.01519638 0.2742059  0.18785866 0.09188785 0.11413821
  0.05642976 0.07376919 0.1

INFO:tensorflow:global_step/sec: 1.56254
INFO:tensorflow:probabilities = [[0.16759387 0.10811664 0.04397309 0.02098895 0.01862889 0.00440796
  0.00814005 0.03204159 0.31519484 0.28091412]
 [0.03214584 0.09710056 0.14160244 0.12752045 0.12765829 0.10594408
  0.19024288 0.10169331 0.04203385 0.0340583 ]
 [0.02060867 0.08439688 0.05380139 0.18003291 0.16135827 0.0915926
  0.27900941 0.06970831 0.03270634 0.02678522]
 [0.02679653 0.024      0.04590703 0.12082332 0.01587283 0.07614017
  0.2084432  0.09016662 0.0505913  0.34125902]
 [0.05493224 0.12461417 0.07459569 0.04902064 0.07250091 0.08946794
  0.0181289  0.04689165 0.23218548 0.2376624 ]
 [0.0108924  0.25420064 0.0075619  0.02699578 0.03394891 0.09583353
  0.25456179 0.0609341  0.08602303 0.16904792]
 [0.10413504 0.22015822 0.04449205 0.0409232  0.01474121 0.02744982
  0.08074982 0.12517074 0.02641815 0.31576175]
 [0.00180883 0.35538338 0.03453471 0.06178003 0.08682746 0.12389388
  0.17416039 0.09654781 0.00329861 0.0617649 ]
 [0.0656

INFO:tensorflow:step = 7502, loss = 1.9594179391860962 (63.998 sec)
INFO:tensorflow:Saving checkpoints for 7526 into /tmp/cfar10_test3/model.ckpt.
INFO:tensorflow:probabilities = [[0.00867719 0.56824262 0.00572251 0.00530701 0.00395459 0.00541877
  0.05582538 0.01154736 0.01762213 0.31768244]
 [0.03234597 0.02221392 0.13784691 0.0158752  0.0268935  0.00611967
  0.16326135 0.56141614 0.02452674 0.00950059]
 [0.06555627 0.05461015 0.13563308 0.19116166 0.09579547 0.07883354
  0.12091956 0.0771191  0.14237255 0.03799861]
 [0.03696113 0.01299109 0.10009179 0.07690895 0.20021487 0.18493262
  0.1008735  0.25704504 0.02325623 0.00672477]
 [0.10701052 0.13118138 0.01298483 0.39526341 0.03357502 0.04621931
  0.1194999  0.13255935 0.01257145 0.00913483]
 [0.03282174 0.52097912 0.00639382 0.00861357 0.01406346 0.0089302
  0.06682035 0.00698659 0.16009915 0.17429201]
 [0.0491478  0.05789427 0.15236562 0.08841706 0.09160134 0.03818343
  0.13590856 0.05522114 0.05649696 0.27476382]
 [0.03057359 0.06

INFO:tensorflow:global_step/sec: 1.56414
INFO:tensorflow:probabilities = [[0.25912536 0.05361565 0.05722499 0.06191194 0.03365291 0.07450488
  0.14938817 0.07078032 0.16895433 0.07084145]
 [0.23005415 0.06758033 0.16259909 0.01891462 0.07315624 0.01170227
  0.15793698 0.02640961 0.09933471 0.15231201]
 [0.04270116 0.00741965 0.01847472 0.15939448 0.02056002 0.4024831
  0.00760016 0.32619504 0.00360743 0.01156423]
 [0.01430929 0.04709692 0.08735768 0.222033   0.14733434 0.17500554
  0.06390418 0.20762969 0.02317175 0.01215761]
 [0.03768635 0.06035543 0.10779176 0.07180564 0.1253179  0.0385274
  0.08206527 0.04738193 0.16687926 0.26218907]
 [0.01870026 0.38236017 0.0243979  0.01773677 0.03066484 0.06314681
  0.06464813 0.08041016 0.0911297  0.22680526]
 [0.00222587 0.81762906 0.00386629 0.02315411 0.00186513 0.00234066
  0.02174774 0.05863721 0.00487833 0.06365561]
 [0.39518648 0.01412627 0.05611091 0.01588142 0.01598272 0.0041574
  0.01991267 0.00646672 0.44942827 0.02274714]
 [0.233012

INFO:tensorflow:step = 7602, loss = 1.9684168100357056 (63.933 sec)
INFO:tensorflow:probabilities = [[0.35961202 0.08388191 0.07905545 0.0350214  0.03621256 0.01164745
  0.08797863 0.10530515 0.09966025 0.10162517]
 [0.09224834 0.24014807 0.0164372  0.02192004 0.00588886 0.01006739
  0.03128103 0.03239869 0.18344301 0.36616736]
 [0.08066716 0.03789632 0.13750313 0.27892851 0.09213333 0.11512559
  0.091089   0.08116316 0.04487347 0.04062033]
 [0.07239608 0.01995023 0.08236177 0.33468578 0.02381382 0.26038368
  0.00307334 0.08656379 0.08897018 0.02780133]
 [0.02479238 0.01342669 0.14188153 0.2351319  0.17571311 0.13470241
  0.11325538 0.11022052 0.03823584 0.01264024]
 [0.01715391 0.1519423  0.05984525 0.07831508 0.02640247 0.03203726
  0.04056579 0.11954744 0.02980646 0.44438403]
 [0.03535895 0.03123837 0.0698102  0.07476727 0.08956751 0.42476277
  0.05144987 0.13701621 0.03146857 0.05456028]
 [0.07458214 0.03101717 0.23119717 0.09206097 0.05170438 0.25729519
  0.06377352 0.09474719 0.0

In [0]:
print (x_train[:,:32*32].shape)

(50000, 1024)
